In [1]:
import glob
import pandas as pd

In [13]:
# 获取当前文件夹下所有的 CSV 文件
file_list = glob.glob('./twitter_roberta/*.csv')

# 用一个空的 DataFrame 开始合并
combined_df = pd.DataFrame()

# 循环读取每个 CSV 文件并合并到 combined_df 中
for file in file_list:
    df = pd.read_csv(file)
    if df.empty:
        continue    
    combined_df =  pd.concat([combined_df, df], ignore_index=True)

In [14]:
combined_df.head()

,Unnamed: 0,sentiment_scores,times
0,0,[-0.2951875943717034],2006-10-20
1,1,[0.0],2006-10-21
2,2,"[0.0, 0.0]",2006-10-23
3,3,"[0.0, 0.0, 0.0, 0.0]",2006-10-24
4,4,"[0.0, 0.0]",2006-10-25


In [15]:
data = combined_df[['sentiment_scores','times']].copy()
print(data.head())
data.describe()

        sentiment_scores       times
0  [-0.2951875943717034]  2006-10-20
1                  [0.0]  2006-10-21
2             [0.0, 0.0]  2006-10-23
3   [0.0, 0.0, 0.0, 0.0]  2006-10-24
4             [0.0, 0.0]  2006-10-25


,sentiment_scores,times
count,1918,1918
unique,1351,1918
top,[0.0],2006-10-20
freq,289,1


In [16]:
data.to_csv('datas_twitter_roberta.csv')

In [9]:
# # 读取月粒度标签
# All_Employees = pd.read_excel('./Ecnomical_index/Monthly All Employees, Total Nonfarm (PAYEMS).xls')
# Employment_Level = pd.read_excel('./Ecnomical_index/Monthly Employment Level.xls')
# Industrial_Production = pd.read_excel('./Ecnomical_index/Monthly Industrial Production Total Index (INDPRO).xls')
# Manufacturing_Trade_Industries_Sales = pd.read_excel('./Ecnomical_index/Monthly Real Manufacturing and Trade Industries Sales (CMRMT).xls')
# Personal_Consumption_Expenditures = pd.read_excel('./Ecnomical_index/Monthly Real Personal Consumption Expenditures (PCEC96).xls')
# personal_income = pd.read_excel('./Ecnomical_index/Monthly Real personal income excluding current transfer receipts (W875RX1).xls')

In [6]:
# 读取季度粒度标签
GDP_GDI = pd.read_excel('./Ecnomical_index/Quarterly Real Average of GDP and GDI (LB0000091Q020SBEA).xls')
GDI = pd.read_excel('./Ecnomical_index/Quarterly Real gross domestic income (A261RX1Q020SBEA).xls')
# GDP = pd.read_excel("./Ecnomical_index/Quarterly Real Gross Domestic Product(GDP).xls") # The datas of GDP has been contained in the Processed_GDP file
Processed_GDP = pd.read_csv('./Ecnomical_index/Quarterly Processed_GDP.csv')


In [9]:
Processed_GDP.head()

,Unnamed: 0,observation_date,GDPC1,InflationCoeff,inflation_GDP,GDP Growth Rate,CAGR,CAGR2
0,56,1961-01-01,3493.703,1.014580,3443.497631,-0.007717,-0.020950,0.600648
1,57,1961-04-01,3553.021,1.014580,3501.963217,0.016979,-0.004327,0.627824
2,58,1961-07-01,3621.252,1.014580,3569.213721,0.019204,0.014794,0.659084
3,59,1961-10-01,3692.289,1.014580,3639.229902,0.019617,0.034701,0.691630
4,60,1962-01-01,3758.147,1.025443,3664.900531,0.007054,0.020784,0.312175


In [10]:
def datetime2index(data,time_clo):
    # 将日期转换为 datetime 类型，并设置为索引
    data[time_clo] = pd.to_datetime(data[time_clo])
    data.set_index(time_clo, inplace=True)
    try:
        data = data.drop("Unnamed: 0",axis=1)
    except:
        return data
    return data

# turn time column into "datetime" index
GDP_GDI = datetime2index(GDP_GDI,'observation_date')
# GDP = datetime2index(GDP,'observation_date') # The datas of GDP has been contained in the Processed_GDP file
GDI = datetime2index(GDI,'observation_date')
Processed_GDP = datetime2index(Processed_GDP,'observation_date')


In [12]:
combined_label = pd.concat([GDP_GDI, GDI, Processed_GDP], axis=1, join='inner')
combined_label.head()

,LB0000091Q020SBEA,A261RX1Q020SBEA,GDPC1,InflationCoeff,inflation_GDP,GDP Growth Rate,CAGR,CAGR2
observation_date,,,,,,,,
1961-01-01,3497.562,3501.421,3493.703,1.014580,3443.497631,-0.007717,-0.020950,0.600648
1961-04-01,3556.780,3560.538,3553.021,1.014580,3501.963217,0.016979,-0.004327,0.627824
1961-07-01,3619.607,3617.962,3621.252,1.014580,3569.213721,0.019204,0.014794,0.659084
1961-10-01,3697.999,3703.710,3692.289,1.014580,3639.229902,0.019617,0.034701,0.691630
1962-01-01,3753.376,3748.605,3758.147,1.025443,3664.900531,0.007054,0.020784,0.312175


In [13]:
combined_label.to_csv('./Quarterly_merged_label.csv')